In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
mc_data = pd.read_json('DataSets/SciFi_200000_Clean_Bool_TrackStudy.json')
# real_data = pd.read_json('DataSets/SciFi_200000_AllHits.json')

In [3]:
###### Data columns
i = 0
for val in mc_data.columns:
    print(i," ",val)
    i = i + 1
# Data columns
# i = 0
# for val in real_data.columns:
#    print(i," ",val)
#    i = i + 1

0   MCParticleID
1   MC_Hit_X
2   MC_Hit_Y
3   MC_Hit_Z
4   PrHit_LHCbID
5   PrHit_Xat0
6   PrHit_Zat0
7   PrHit_isX
8   PrHit_planeCode
9   PrHit_w2
10   PrHit_yMax
11   PrHit_yMin
12   PrHit_zone
13   event
14   hasT
15   isInUT
16   isInVelo
17   isLong
18   isLong_fromB
19   isLong_fromB_more5
20   isLong_more5


In [4]:
mc_hits = mc_data.values
# real_hits = real_data.values

In [55]:
zLays = [7825.99951172, 7895.99951172, 7966.        , 8036.00048828,
       8507.99902344, 8577.99902344, 8647.99902344, 8718.00097656,
       9192.99902344, 9262.99902344, 9332.99902344, 9403.00097656]

max_x = 3182.8356933594
min_x = -3182.8576660156

In [94]:
# Create X and Y matrices for the future model
length = mc_hits.shape[0]

x_length = 1
y_length = 1

X = np.zeros((length, x_length))
Y = np.zeros((length, y_length))
invZfirst = 1.0 / (zLays[0])

i = 0
empty_particles = []
for i in range(0,length):
    
    if (len(mc_hits[i][5]) > 1) and (mc_hits[i][12][0] == 0):
        # Get the first element of the hit, in the first layer
        X[i][0] = mc_hits[i][5][0]
        
        # Get only the first element if there are several repeated with small differences
        j = 0
        extra_hits = mc_hits[i][5]
        try:
            while (np.abs(extra_hits[j] - extra_hits[j + 1]) < 5):
                j += 1
            j += 1
        except IndexError as e:
            X[i] = np.zeros(x_length)
            continue
        
        tx_inf = (X[i][0] * invZfirst)
        xProjeInf = tx_inf * zLays[11]
        Y[i][0] = mc_hits[i][5][len(mc_hits[i][5]) - 1]
        print("X1: %d X2: %d Pred: %d" % (X[i][0],Y[i][0],xProjeInf))
        # Get all hits from the given particle which are not located in the first layer
        # extra_hits = mc_hits[i][5][j:]
        # actual_pos = 0
        # j = 0
        #try:
        #    while (j < len(extra_hits)):
        #        Y[i][actual_pos] = extra_hits[j]
        #        # Same as above, get only the first element in case there are several hits with similar X position
        #        while (j + 1 < len(extra_hits) and np.abs(extra_hits[j] - extra_hits[j + 1]) < 3):
        #            j += 1
        #        actual_pos += 1
        #        j += 1
        #except IndexError:
        #    X[i] = np.zeros(x_length)
        #    Y[i] = np.zeros(y_length)
            
# Removing empty values, as some hits in the mc_hits dataset do not contain readings (PrHit...)
Y = Y[Y[:,0] != 0]
X = X[X[:,0] != 0]

X1: -795 X2: -2576 Pred: -956
X1: 232 X2: 383 Pred: 279
X1: 453 X2: 852 Pred: 544
X1: -775 X2: -2221 Pred: -931
X1: -880 X2: -2529 Pred: -1057
X1: 373 X2: 859 Pred: 448
X1: -358 X2: -732 Pred: -430
X1: 1406 X2: 2369 Pred: 1690
X1: 315 X2: 491 Pred: 379
X1: -380 X2: -539 Pred: -457
X1: 1133 X2: 1848 Pred: 1361
X1: -482 X2: -648 Pred: -579
X1: -1762 X2: -2623 Pred: -2117
X1: -283 X2: -312 Pred: -340
X1: -1211 X2: -1320 Pred: -1455
X1: 1242 X2: 1668 Pred: 1492
X1: 1481 X2: 1896 Pred: 1780
X1: 141 X2: -13 Pred: 170
X1: 266 X2: 484 Pred: 320
X1: 749 X2: 1519 Pred: 901
X1: 854 X2: 1388 Pred: 1027
X1: 1039 X2: 1940 Pred: 1248
X1: 1411 X2: 2311 Pred: 1696
X1: 46 X2: 381 Pred: 55
X1: 254 X2: 591 Pred: 305
X1: -69 X2: 291 Pred: -83
X1: -477 X2: -368 Pred: -573
X1: -987 X2: -1730 Pred: -1186
X1: 1716 X2: 3054 Pred: 2062
X1: 380 X2: 791 Pred: 457
X1: -719 X2: -243 Pred: -864
X1: 138 X2: 301 Pred: 166
X1: 893 X2: 1174 Pred: 1073
X1: -167 X2: -287 Pred: -200
X1: 138 X2: 325 Pred: 166
X1: 322 X2: 114

X1: -5 X2: -104 Pred: -7
X1: 753 X2: 1128 Pred: 905
X1: -70 X2: -58 Pred: -84
X1: -475 X2: -849 Pred: -571
X1: -444 X2: -1047 Pred: -534
X1: -36 X2: 32 Pred: -43
X1: 13 X2: 129 Pred: 15
X1: 882 X2: 2265 Pred: 1060
X1: -104 X2: -205 Pred: -125
X1: 472 X2: 761 Pred: 567
X1: 62 X2: 313 Pred: 75
X1: 211 X2: 990 Pred: 254
X1: -250 X2: -1329 Pred: -301
X1: -153 X2: -280 Pred: -184
X1: -586 X2: -784 Pred: -704
X1: -269 X2: -195 Pred: -323
X1: 38 X2: 74 Pred: 46
X1: -189 X2: -283 Pred: -227
X1: 234 X2: 437 Pred: 281
X1: -150 X2: -258 Pred: -181
X1: -1730 X2: -2890 Pred: -2079
X1: 1061 X2: 1683 Pred: 1275
X1: 1262 X2: 1710 Pred: 1516
X1: 134 X2: 282 Pred: 161
X1: 1520 X2: 2249 Pred: 1827
X1: -162 X2: -1644 Pred: -194
X1: -228 X2: -562 Pred: -274
X1: -1825 X2: -3116 Pred: -2192
X1: 142 X2: 1040 Pred: 171
X1: 11 X2: -22 Pred: 14
X1: 243 X2: 561 Pred: 292
X1: 113 X2: 146 Pred: 136
X1: -600 X2: -1023 Pred: -721
X1: -135 X2: -352 Pred: -162
X1: 1043 X2: 1811 Pred: 1253
X1: -157 X2: -224 Pred: -189
X

X1: -298 X2: -276 Pred: -358
X1: 981 X2: 1576 Pred: 1178
X1: 704 X2: 1020 Pred: 846
X1: -147 X2: 19 Pred: -177
X1: -1085 X2: -2851 Pred: -1304
X1: 401 X2: 682 Pred: 482
X1: -1494 X2: -2424 Pred: -1795
X1: -386 X2: -1819 Pred: -464
X1: -382 X2: -606 Pred: -459
X1: 319 X2: 732 Pred: 384
X1: -721 X2: -1164 Pred: -866
X1: -930 X2: -1290 Pred: -1117
X1: 198 X2: 177 Pred: 238
X1: 198 X2: 241 Pred: 238
X1: 1561 X2: 2668 Pred: 1876
X1: 359 X2: 1459 Pred: 431
X1: -448 X2: -1243 Pred: -539
X1: 66 X2: 177 Pred: 79
X1: -958 X2: -2770 Pred: -1152
X1: 229 X2: 319 Pred: 275
X1: 584 X2: 918 Pred: 701
X1: 1276 X2: 2215 Pred: 1534
X1: -617 X2: -985 Pred: -741
X1: -793 X2: -1466 Pred: -953
X1: 31 X2: 325 Pred: 37
X1: 31 X2: -622 Pred: 37
X1: 205 X2: 177 Pred: 247
X1: -774 X2: -1754 Pred: -930
X1: -635 X2: -1424 Pred: -763
X1: 1337 X2: 156 Pred: 1606
X1: -303 X2: -2823 Pred: -365
X1: -181 X2: -985 Pred: -218
X1: -550 X2: -1160 Pred: -660
X1: 205 X2: 398 Pred: 246
X1: 123 X2: -70 Pred: 147
X1: -185 X2: -43

X1: 217 X2: 724 Pred: 261
X1: 1649 X2: 2444 Pred: 1981
X1: -249 X2: -451 Pred: -299
X1: 1952 X2: 3155 Pred: 2346
X1: 407 X2: 632 Pred: 489
X1: 1034 X2: 1532 Pred: 1242
X1: 440 X2: 597 Pred: 529
X1: -69 X2: 943 Pred: -83
X1: -587 X2: -1748 Pred: -705
X1: 622 X2: 1336 Pred: 747
X1: -1536 X2: -3151 Pred: -1846
X1: -1399 X2: -1958 Pred: -1681
X1: -216 X2: -711 Pred: -259
X1: -269 X2: 324 Pred: -324
X1: 1412 X2: 2041 Pred: 1696
X1: 692 X2: 1316 Pred: 831
X1: -293 X2: -406 Pred: -352
X1: 1862 X2: 2659 Pred: 2238
X1: -390 X2: -740 Pred: -469
X1: -1968 X2: -2959 Pred: -2365
X1: -1444 X2: -2394 Pred: -1735
X1: -1631 X2: -2306 Pred: -1960
X1: -1138 X2: -1632 Pred: -1368
X1: 1619 X2: 2768 Pred: 1945
X1: -282 X2: 556 Pred: -339
X1: -911 X2: -1652 Pred: -1095
X1: -227 X2: -400 Pred: -273
X1: 151 X2: 200 Pred: 181
X1: -457 X2: -774 Pred: -550
X1: -173 X2: -319 Pred: -208
X1: -160 X2: -258 Pred: -192
X1: 28 X2: 14 Pred: 33
X1: 138 X2: 269 Pred: 166
X1: -174 X2: -1518 Pred: -209
X1: 313 X2: 911 Pred: 

X1: -746 X2: 590 Pred: -897
X1: -747 X2: -568 Pred: -898
X1: -1392 X2: -1183 Pred: -1673
X1: -1392 X2: -1717 Pred: -1673
X1: 332 X2: 744 Pred: 399
X1: 169 X2: 352 Pred: 203
X1: 29 X2: -270 Pred: 35
X1: -630 X2: 272 Pred: -757
X1: 1411 X2: 2048 Pred: 1696
X1: 285 X2: 403 Pred: 342
X1: -329 X2: -516 Pred: -395
X1: -223 X2: -796 Pred: -268
X1: 174 X2: 388 Pred: 209
X1: -763 X2: -1065 Pred: -916
X1: 217 X2: 405 Pred: 261
X1: -1973 X2: -2923 Pred: -2371
X1: -1160 X2: -2678 Pred: -1394
X1: 293 X2: 350 Pred: 352
X1: -168 X2: -273 Pred: -201
X1: 1079 X2: 1801 Pred: 1296
X1: -149 X2: -333 Pred: -179
X1: -1744 X2: -2927 Pred: -2096
X1: 167 X2: 3159 Pred: 201
X1: 159 X2: 2799 Pred: 191
X1: -145 X2: -470 Pred: -175
X1: -21 X2: -262 Pred: -25
X1: 1071 X2: 1088 Pred: 1287
X1: -192 X2: -1249 Pred: -230
X1: 617 X2: 881 Pred: 741
X1: 17 X2: 48 Pred: 20
X1: -769 X2: -1259 Pred: -924
X1: 1297 X2: 2649 Pred: 1559
X1: 1500 X2: 2752 Pred: 1803
X1: 859 X2: 1494 Pred: 1033
X1: 413 X2: -12 Pred: 496
X1: 1908 X

...

X1: -517 X2: -819 Pred: -622
X1: -134 X2: -521 Pred: -161
X1: 253 X2: 523 Pred: 304
X1: -182 X2: -721 Pred: -219
X1: -207 X2: -1167 Pred: -249
X1: -541 X2: -910 Pred: -650
X1: -184 X2: -721 Pred: -221
X1: -146 X2: -2025 Pred: -175
X1: -260 X2: -1393 Pred: -313
X1: -177 X2: -748 Pred: -212
X1: 178 X2: 142 Pred: 213
X1: 178 X2: 313 Pred: 213
X1: 435 X2: 1569 Pred: 523
X1: -283 X2: -1326 Pred: -340
X1: -277 X2: -427 Pred: -333
X1: 196 X2: 2369 Pred: 235
X1: 135 X2: 948 Pred: 162
X1: -221 X2: -1157 Pred: -265
X1: 1274 X2: 2198 Pred: 1531
X1: -950 X2: -1597 Pred: -1141
X1: -379 X2: -2202 Pred: -456
X1: -579 X2: -987 Pred: -696
X1: 600 X2: 503 Pred: 721
X1: 1515 X2: 2565 Pred: 1821
X1: -173 X2: -553 Pred: -208
X1: 500 X2: 1296 Pred: 600
X1: -163 X2: -221 Pred: -196
X1: -276 X2: -435 Pred: -331
X1: -760 X2: -1581 Pred: -913
X1: -1276 X2: -2143 Pred: -1533
X1: -298 X2: -834 Pred: -358
X1: -1091 X2: -2352 Pred: -1311
X1: -144 X2: -470 Pred: -173
X1: 1711 X2: 2733 Pred: 2056
X1: -240 X2: -1654 P

X1: 62 X2: 111 Pred: 75
X1: 296 X2: 1481 Pred: 356
X1: 144 X2: 294 Pred: 173
X1: 332 X2: 523 Pred: 399
X1: 1184 X2: 1826 Pred: 1423
X1: -1572 X2: -2436 Pred: -1889
X1: -1688 X2: -2502 Pred: -2028
X1: -511 X2: -811 Pred: -615
X1: 1873 X2: 2933 Pred: 2251
X1: -877 X2: -1523 Pred: -1054
X1: -1423 X2: -2338 Pred: -1710
X1: 1430 X2: 2497 Pred: 1718
X1: 110 X2: -177 Pred: 132
X1: 247 X2: 878 Pred: 297
X1: 53 X2: 913 Pred: 64
X1: 1122 X2: 446 Pred: 1348
X1: 78 X2: 719 Pred: 94
X1: 975 X2: 2082 Pred: 1172
X1: -723 X2: -952 Pred: -869
X1: 265 X2: 421 Pred: 318
X1: -70 X2: -124 Pred: -84
X1: -149 X2: -281 Pred: -179
X1: 300 X2: 423 Pred: 361
X1: -155 X2: -313 Pred: -187
X1: -165 X2: -296 Pred: -199
X1: 172 X2: 225 Pred: 207
X1: 228 X2: 1765 Pred: 274
X1: -121 X2: 934 Pred: -145
X1: -282 X2: -680 Pred: -339
X1: 381 X2: -89 Pred: 458
X1: -825 X2: -1319 Pred: -992
X1: 1552 X2: 2231 Pred: 1865
X1: -97 X2: -219 Pred: -116
X1: -143 X2: -775 Pred: -171
X1: -20 X2: 101 Pred: -24
X1: 1037 X2: 1567 Pred: 

X1: -358 X2: -595 Pred: -431
X1: 205 X2: 303 Pred: 246
X1: -174 X2: -808 Pred: -209
X1: -479 X2: -763 Pred: -576
X1: -1009 X2: -1389 Pred: -1213
X1: 65 X2: 1196 Pred: 79
X1: 1499 X2: 2728 Pred: 1801
X1: -456 X2: -612 Pred: -549
X1: 201 X2: 633 Pred: 242
X1: 280 X2: 596 Pred: 336
X1: -123 X2: -78 Pred: -148
X1: -58 X2: -643 Pred: -70
X1: 1081 X2: 2182 Pred: 1299
X1: -1325 X2: -380 Pred: -1592
X1: -2492 X2: -2707 Pred: -2995
X1: -291 X2: -458 Pred: -350
X1: -1079 X2: -1744 Pred: -1296
X1: 661 X2: 919 Pred: 794
X1: 1375 X2: 1894 Pred: 1652
X1: -1560 X2: -2564 Pred: -1874
X1: -1075 X2: -1494 Pred: -1292
X1: 1316 X2: 2218 Pred: 1581
X1: -108 X2: -378 Pred: -130
X1: -1192 X2: -1602 Pred: -1432
X1: -350 X2: -552 Pred: -421
X1: -856 X2: -1373 Pred: -1028
X1: -42 X2: -93 Pred: -51
X1: 1099 X2: 1721 Pred: 1320
X1: 140 X2: 472 Pred: 168
X1: -646 X2: -1416 Pred: -777
X1: -779 X2: -1205 Pred: -935
X1: 1629 X2: 2770 Pred: 1958
X1: -2583 X2: -1896 Pred: -3104
X1: -150 X2: -159 Pred: -181
X1: -136 X2:

X1: -484 X2: -748 Pred: -582
X1: 1289 X2: 1787 Pred: 1548
X1: 1155 X2: 1664 Pred: 1387
X1: -191 X2: -312 Pred: -230
X1: 239 X2: 388 Pred: 287
X1: -101 X2: -309 Pred: -122
X1: 778 X2: 2064 Pred: 935
X1: -1683 X2: -1410 Pred: -2022
X1: 171 X2: 243 Pred: 205
X1: -1290 X2: -2052 Pred: -1550
X1: 924 X2: 2318 Pred: 1110
X1: 924 X2: 355 Pred: 1110
X1: 924 X2: 1703 Pred: 1110
X1: 2101 X2: 2929 Pred: 2524
X1: -1274 X2: -2703 Pred: -1531
X1: -213 X2: -509 Pred: -256
X1: -138 X2: -272 Pred: -166
X1: -574 X2: -904 Pred: -690
X1: -755 X2: -1082 Pred: -908
X1: -1293 X2: -1322 Pred: -1553
X1: 834 X2: 1339 Pred: 1003
X1: -1278 X2: -2032 Pred: -1536
X1: -141 X2: -227 Pred: -170
X1: -141 X2: -141 Pred: -170
X1: 430 X2: 1136 Pred: 517
X1: 1040 X2: 1837 Pred: 1250
X1: 810 X2: 1224 Pred: 974
X1: 314 X2: 441 Pred: 377
X1: -1726 X2: -2769 Pred: -2074
X1: 1010 X2: 1400 Pred: 1213
X1: 1775 X2: 2731 Pred: 2132
X1: -262 X2: -376 Pred: -315
X1: 502 X2: 803 Pred: 603
X1: -499 X2: -817 Pred: -600
X1: 799 X2: 1484 P

X1: -483 X2: -1057 Pred: -581
X1: -1310 X2: -2189 Pred: -1574
X1: -346 X2: -114 Pred: -416
X1: 550 X2: 1002 Pred: 661
X1: 387 X2: 626 Pred: 465
X1: 378 X2: 654 Pred: 454
X1: 88 X2: -328 Pred: 105
X1: 248 X2: 418 Pred: 298
X1: 207 X2: -408 Pred: 248
X1: 1661 X2: 2463 Pred: 1996
X1: -1074 X2: -2334 Pred: -1290
X1: 347 X2: 468 Pred: 417
X1: -1554 X2: -2622 Pred: -1867
X1: -949 X2: -2250 Pred: -1140
X1: 434 X2: 1142 Pred: 522
X1: 405 X2: 24 Pred: 486
X1: -1387 X2: -2200 Pred: -1666
X1: -1805 X2: -3137 Pred: -2168
X1: 161 X2: 2675 Pred: 194
X1: 707 X2: 1142 Pred: 849
X1: -624 X2: -1431 Pred: -750
X1: 353 X2: 617 Pred: 424
X1: 690 X2: 1183 Pred: 829
X1: -1712 X2: -3084 Pred: -2057
X1: -398 X2: -583 Pred: -478
X1: -384 X2: -583 Pred: -462
X1: -924 X2: -584 Pred: -1110
X1: -194 X2: -255 Pred: -233
X1: 1424 X2: 2854 Pred: 1711
X1: -1196 X2: -1994 Pred: -1437
X1: 595 X2: 928 Pred: 715
X1: -178 X2: -284 Pred: -214
X1: -688 X2: -1036 Pred: -827
X1: -219 X2: -456 Pred: -263
X1: -752 X2: -1416 Pred:

In [85]:
X.shape

(70867, 1)

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

model = Sequential()
model.add(Dense(12, input_dim=1, kernel_initializer='normal', activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
model.summary()
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

history = model.fit(X_train, y_train, epochs=150, batch_size=50,  verbose=1, validation_split=0.2)

print(history.history.keys())
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_69 (Dense)             (None, 12)                24        
_________________________________________________________________
dense_70 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_71 (Dense)             (None, 1)                 9         
Total params: 137
Trainable params: 137
Non-trainable params: 0
_________________________________________________________________
Train on 42520 samples, validate on 10630 samples
Epoch 1/150
42520/42520 [==============================] - 3s 64us/sample - loss: 682424.4416 - mean_squared_error: 682423.8750 - mean_absolute_error: 520.7571 - val_loss: 292412.6834 - val_mean_squared_error: 292412.6562 - val_mean_absolute_error: 309.8244
Epoch 2/150
42520/42520 [==============================] - 2s 53us/sample - loss: 293108.8289 - mean

KeyboardInterrupt: 

In [13]:
def showPredictedTrack(X, Y):
    fig = plt.figure(figsize=(19, 8))
    ax = fig.add_subplot(111)
    ax.plot(X, Y)       
    ax.grid()
    plt.show()